## Clean Water Act: Violations 
### By Congressional District

This notebook examines ECHO data on the National Pollutant Discharge Elimination System, or NPDES, which was established under the Clean Water Act to require monitoring and compliance from wastewater treatment plants, factories, and other point sources of water pollution. The notebook uses NPDES_QNCR_HISTORY, 
which reports the number of violations in four categories:

1. Effluent (E90) violations: `NUME90Q`
2. Compliance schedule violations:`NUMCVDT`
3. Single event violations: `NUMSVCD`
4. Permit schedule violations: `NUMPSCH`

These fields are used from the ECHO_EXPORTER data set:
- `REGISTRY_ID`: a unique facility identifier
- `FAC_NAME`: facility name
- `FAC_STATE`
- `NPDES_IDS`: a list of ids that match NPDES_ID in other data sets
- `FAC_LAT`: facility's latitude
- `FAC_LONG`: longitude
- `FAC_DERIVED_CD113`: 113th Congressional District

A state and congressional district must be chosen using the dropdown widgets that are provided.

---
---

## How to Run
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* **It is important to run cells in order because they depend on each other.**
* Some cells, like the one shown below, will create a dropdown menu after you run them. Be sure to make a selection (for example, click to change NY to LA) before running the next cell.
![Dropdown menu](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/dropdown.JPG?raw=true)
* Other cells will simply print information when you run them, like this one:
![Simple cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/cell-simple.JPG?raw=true)
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---
---

# Let's begin! 
### Hover over the circle on the top left corner of the cell below and you should see a "play" button appear. Click on it to run the cell. 
Doing so will load in some extra code to help us make sense of our ECHO data and when it finishes, you should see your cell grayed out. You can now move on to the next one.

In [ ]:
# Import libraries
import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display


### Run this next cell to create a widget for selecting states. It will create a dropdown menu at the bottom. Choose your state from the menu then move on to the next cell.

In [ ]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
dropdown_state=widgets.Dropdown(
    options=states,
    value='NY',
    description='State:',
    disabled=False,
)
output_state = widgets.Output()
my_state = ""

def dropdown_state_eventhandler( change ):
    output_state.clear_output()
    value = change.new
    with output_state:
        display( change.new )
            
dropdown_state.observe( dropdown_state_eventhandler, names='value')
display( dropdown_state )


### Run this cell after choosing a state. It will pull the data for that state from ECHO

In [ ]:
my_state = dropdown_state.value

sql = "select REGISTRY_ID, FAC_NAME, FAC_STATE, NPDES_IDS, FAC_LAT, FAC_LONG, FAC_DERIVED_CD113 " + \
    " from ECHO_EXPORTER " + \
    " where NPDES_FLAG = 'Y' and FAC_STATE = '" + my_state + "'"
url='http://apps.tlt.stonybrook.edu/echoepa/?query='
data_location=url+urllib.parse.quote(sql)
print(sql)
# print(data_location)



### Run this cell to load the CSV of that data.

In [ ]:
echo_data = pd.read_csv(data_location,encoding='iso-8859-1',header = 0)
echo_data.set_index( 'REGISTRY_ID', inplace=True )


### How many facilities in the selected state are tracked for water pollution under CWA?

In [ ]:
num_facilities = echo_data.shape[0]
print("There are %s NDIS facilities in %s tracked in the ECHO database." %(num_facilities, my_state))


### Run this next cell to generate the Congressional District dropdown list for your state. 

#### Here is a map of congressional districts: https://www.govtrack.us/congress/members/map

In [ ]:
if (( my_state != 'none' ) & (my_state != 'all' )):
    cd_array = echo_data["FAC_DERIVED_CD113"].fillna(0).astype(int).unique()
    cd_array.sort()
    w2=widgets.Dropdown(
        options=cd_array,
        value=1,
        description='Congressional Districts:',
        disabled=False,
    )
    display(w2)


### Select a CD and run the following cell:

In [ ]:
my_cd = w2.value
my_cd_facs = echo_data[echo_data["FAC_DERIVED_CD113"].fillna(0).astype(int) == my_cd]
num_facilities = my_cd_facs.shape[0]    
print("There are %s NDIS facilities in %s district %s tracked in the ECHO database." %(num_facilities, my_state, my_cd))


### Next look up the violation history for the facilities in the selected state and congressional district. This step may take a while to run. What we'll get back is a list of facility IDs and their dates of violations.
How many are there? Below the table, the number of rows listed is the total number of CWA violations that have occurred over the history of the district since they started tracking in this database.

In [ ]:
# The NPDES_ID is of the form GAISO1239, with the first two characters being the state abbreviation.
# Use that in the query to reduce the number of records and speed up the execution.

sql = "select NPDES_ID, YEARQTR, NUME90Q, NUMCVDT, NUMSVCD, NUMPSCH" + \
    " from NPDES_QNCR_HISTORY where YEARQTR > 20050 and YEARQTR < 20200 and " \
    " NPDES_ID like '" + my_state + "%'"
url='http://apps.tlt.stonybrook.edu/echoepa/?query='
data_location=url+urllib.parse.quote(sql)
# print(sql)

qncr_data = pd.read_csv(data_location,header = 0)
qncr_data.set_index( "NPDES_ID", inplace=True)
qncr_data


### This cell gets more information about each facility. Run it to set up for the next part.

In [ ]:
# The NPDES_IDS in ECHO_EXPORTER can contain multiple ids for a facility. 
# The string must be parsed to get each individual NPDES_ID to look up 
# in NPDES_QNCR_HISTORY.

my_cd_npdes = pd.DataFrame()
no_data_ids = []
for fac in my_cd_facs.itertuples():
    ids = fac.NPDES_IDS
    for npdes_id in ids.split():
        try:
            npdes_data = qncr_data.loc[ npdes_id ].copy()
            # Add the facility's index number to air_data, to refer to it.
            n = npdes_data.shape[0]
            fac_list = [fac.Index] * n
            npdes_data['facility'] = fac_list
            frames = [my_cd_npdes, npdes_data]
            my_cd_npdes = pd.concat( frames )
        except KeyError:
            no_data_ids.append( npdes_id )



### Ths one is set up too, generating a table that maps facilities to their locations and permit types.

In [ ]:
my_cd_groups = my_cd_npdes.groupby( 'YEARQTR' )[['NUMCVDT','NUME90Q','NUMPSCH','NUMSVCD']].sum()

Here are the different kinds of violations possible under the CWA:

`NUMCVDT` - (Number of Compliance Schedule Violations in Quarter) A count of the number of compliance schedule violations reported in the quarter, defined by YEARQTR.

`NUME90Q` - (Number of E90 Violations in Quarter) A count of the number of effluent violations (E90) reported in the quarter, defined by YEARQTR.

`NUMPSCH` - (Number of Permit Schedule Violations in Quarter) A count of the number of permit schedule violations reported in the quarter, defined by YEARQTR.

`NUMSVCD` - (Number of Single Event Violations in Quarter) A count of the number of single event violations reported in the quarter, defined by PRHQRTR.

### Plot the number of violations by year for the entire congressional district:

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
my_cd_groups.plot(ax=ax)


### Let's show a quick map of your area and the facilities in it. 
#### Once you run this cell, a map should appear. You can zoom in and out, or click on facilities to get their names.

In [ ]:
def mapper(df):
    # Initialize the map
    m = folium.Map(
        location = [df.mean()["FAC_LAT"], df.mean()["FAC_LONG"]],
        zoom_start = 11
    )

    # Add a clickable marker for each facility
    for index, row in df.iterrows():
        folium.Marker(
            location = [row["FAC_LAT"], row["FAC_LONG"]],
            popup = row["FAC_NAME"] ).add_to(m)

    # Show the map
    return m

map_of_facilities_in_my_area = mapper(my_cd_facs)
map_of_facilities_in_my_area


### What if we wanted to focus on just one facility? 
#### First, let's add up all the compliance violations, by facility.

In [ ]:
cd_npdes_group = my_cd_npdes.groupby( 'facility', as_index=False )[['NUMCVDT','NUME90Q','NUMPSCH','NUMSVCD']].sum()


#### Which facility has had the largest number of E90 violations, summed over the entire period since 2005?  
Remember that `NUME90Q` - (Number of E90 Violations in Quarter) A count of the number of effluent violations (E90) reported in the quarter, defined by YEARQTR.

In [ ]:
fac_maxE90 = cd_npdes_group.loc[cd_npdes_group['NUME90Q'].idxmax()]
print(cd_npdes_group.loc[ cd_npdes_group['facility'] == fac_maxE90['facility']])
fac_maxE90_df = my_cd_facs.loc[ my_cd_facs.index == fac_maxE90['facility']]


#### Where is the facility with the most violations since 2005?

In [ ]:
map_of_maxE90 = mapper(fac_maxE90_df)
map_of_maxE90


#### What does this facility's history look like?

In [ ]:
my_cd_npdes.loc[ my_cd_npdes['facility'] == fac_maxE90['facility']]


### This final section saves some of this data to CSV files in your Google Drive.
The first of the next three cells will open our Google Drive to write into.
The second cell writes the congressional district file.
The third cell writes the file for state data. 
**Running these cells is optional.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Write the congressional district data to CSV file.

In [ ]:
filename = '/content/drive/My Drive/cd-cwa-viol-' + my_state + '-' + str( my_cd ) + '.csv'
my_cd_npdes.to_csv( filename ) 
print( "Writing this data to %s" %(filename))


#### Write the state data to CSV file.

In [ ]:
filename = '/content/drive/My Drive/state-cwa-viol-' + my_state + '.csv'
qncr_data.to_csv( filename ) 
print( "Writing this data to %s" %(filename))